In [1]:
import json, os, ensembl_rest

In [2]:
ensembl_rest.lookup(id='ENSG00000139618')['display_name']

'BRCA2'

In [3]:
amigo_dir = '/Users/adrian/gd15/hi/research/osteo/results/enrichment_volcano/json/'

In [4]:
files = os.listdir(amigo_dir)
json_files = [element for element in files if 'json' in element and element[0] != '.']
print(json_files)

['other_down.json', 'other_up.json', 'metabolic_up.json', 'tf_up.json', 'tf_down.json', 'metabolic_down.json']


# analysis

In [5]:
def analysis(json_file):
    
    pattern = json_file.split('.')[0]
    amigo_file = amigo_dir+'{}.json'.format(pattern)
    output_file = amigo_dir+'{}_formatted.tsv'.format(pattern)
    
    g = open(output_file,'w')
    g.write('Level\tTerm\tBackground rank\tFound rank\tExpected rank\tFold enrichment\tSign\tP-value\tENSEMBL IDs\tGene symbols\n')

    with open(amigo_file,'r') as f:
        data = json.load(f)
        for group in data['overrepresentation']['group']:
            if type(group) == dict:
                if type(group['result']) == list:
                    pathways=group['result']
                elif type(group['result']) == dict:
                    pathways=[group['result']]
                else:
                    print('ERROR: value not considered for variable group[result].')
                
                for element in pathways:
                        
                    sign=element['input_list']['plus_minus']
                    if sign == '+':
                        level=element['term']['level']
                        term=element['term']['label']
                        background_rank=element['number_in_reference']
                        found_rank=element['input_list']['number_in_list']
                        expected_rank=element['input_list']['expected']
                        fold_enrichment=element['input_list']['fold_enrichment']
                        pvalue=element['input_list']['pValue']

                        working_variable=element['input_list']['mapped_id_list']['mapped_id']
                        if type(working_variable) == list:
                            ensemblIDs = working_variable
                            gene_symbols = [ensembl_rest.lookup(id=ID)['display_name'] for ID in ensemblIDs]
                        
                        else:
                            print('ERROR: value not considered for variable working_variable.')

                        ensembl_string=', '.join(ensemblIDs)
                        gene_symbols_string=', '.join(gene_symbols)

                        g.write('{}\t'.format(level))
                        g.write('{}\t'.format(term))
                        g.write('{}\t'.format(background_rank))
                        g.write('{}\t'.format(found_rank))
                        g.write('{}\t'.format(expected_rank))
                        g.write('{}\t'.format(fold_enrichment))
                        g.write('{}\t'.format(sign))
                        g.write('{}\t'.format(pvalue))
                        g.write('{}\t'.format(ensembl_string))
                        g.write('{}\t'.format(gene_symbols_string))

                        g.write('\n')
                     
    g.close()
    
    return None

In [6]:
%%time

for element in json_files:
    print('working on {}...'.format(element))
    analysis(element)

working on other_down.json...
working on other_up.json...
working on metabolic_up.json...
working on tf_up.json...
working on tf_down.json...
working on metabolic_down.json...
CPU times: user 13.2 s, sys: 1.36 s, total: 14.5 s
Wall time: 3min 59s
